In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import scipy.io
import time
import pandas as pd

def get_signal(path,num):
    mdata = pd.read_csv(path, header=None, names=["No.", "Value"])
    mear_data = mdata["Value"].values.reshape(-1, 1)
    real = mear_data[0::4, :] - mear_data[1::4, :]
    imag = mear_data[2::4, :] - mear_data[3::4, :]
    real = real[0:num]
    imag = imag[0:num]
    real = torch.from_numpy(real[None, ...])
    imag = torch.from_numpy(imag[None, ...])
    real = torch.unsqueeze(real, -1)
    imag = torch.unsqueeze(imag, -1)
    
    real = (real - torch.mean(real)) / torch.std(real)
    imag = (imag - torch.mean(imag)) / torch.std(imag)

    return real,imag

def get_random_pattern(rng,num,shape,th=0.5):

    p = rng.random((num, 1, shape[0], shape[1]))
    p_pos = (p > th).astype(np.float64)
    p_neg = 1 - p_pos
    p = p_pos - p_neg
    
    return torch.from_numpy(p)

def get_signal1(final_real, p):
    signal_real = F.conv2d(final_real[:, 0:1, :, :], p, stride=1, padding=0)
    signal_real = (signal_real - torch.mean(signal_real)) / torch.std(signal_real)
    
    return signal_real

def get_signal2(final_imag, p):
    signal_imag = F.conv2d(final_imag[:, 0:1, :, :], p, stride=1, padding=0)
    signal_imag = (signal_imag - torch.mean(signal_imag)) / torch.std(signal_imag)
    return signal_imag

def save_list_to_csv(data_list, file_name):
    df = pd.DataFrame(data_list, columns=[file_name.rstrip(".csv")])
    df.to_csv(file_name, index=False)
    print(f"Data saved to {file_name}")
    
def phase_shift(final_real,final_imag):
    real_image = final_real[0, 0, :, :].detach().cpu().numpy()
    imag_image = final_imag[0, 0, :, :].detach().cpu().numpy()
    delta_phi_array = np.arange(0, 3.05, 0.1)
    for data_phi in delta_phi_array:
        
        print(data_phi)
        fn = real_image + 1j * imag_image
    
        fn = fn * np.exp(1j * data_phi)
        amplitude = np.abs(fn)
        phase = np.angle(fn)
    
        plt.figure(figsize=(12, 3))
    
        plt.subplot(1, 4, 1)
        plt.imshow(real_image)
        plt.axis('off')

        plt.subplot(1, 4, 2)
        plt.imshow(imag_image)
        plt.axis('off')

        plt.subplot(1, 4, 3)
        plt.imshow(amplitude)
        plt.axis('off')

        plt.subplot(1, 4, 4)
        plt.imshow(phase)
        plt.axis('off')

        plt.subplots_adjust(wspace=0.15, hspace=0.15)
        plt.show()
        

